In [1]:
import pandas as pd
import numpy as np
import re
import nltk
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
train_data = pd.read_csv("twitter_training.csv")
val_data = pd.read_csv("twitter_validation.csv")

In [3]:
train_data.columns = ['ID', 'Category', 'Sentiment', 'Tweet']
val_data.columns = ['ID', 'Category', 'Sentiment', 'Tweet']

In [4]:
train_data.dropna(subset=['Tweet'], inplace=True)
val_data.dropna(subset=['Tweet'], inplace=True)

In [5]:
nltk.download('stopwords')
stop_words = set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package stopwords to C:\Users\Kunal
[nltk_data]     Tilaganji\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [6]:
def preprocess_text(text):
    text = text.lower()  
    text = re.sub(r'http\S+', '', text)  
    text = re.sub(r'@\w+', '', text)  
    text = re.sub(r'#\w+', '', text) 
    text = re.sub(r'[^\w\s]', '', text) 
    text = " ".join([word for word in text.split() if word not in stop_words])
    return text

In [7]:
train_data['cleaned_text'] = train_data['Tweet'].apply(preprocess_text)
val_data['cleaned_text'] = val_data['Tweet'].apply(preprocess_text)

In [8]:
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(train_data['Sentiment'])
y_val = label_encoder.transform(val_data['Sentiment'])

In [9]:
tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(train_data['cleaned_text'])

X_train = tokenizer.texts_to_sequences(train_data['cleaned_text'])
X_val = tokenizer.texts_to_sequences(val_data['cleaned_text'])

In [10]:
max_length = 50  
X_train = pad_sequences(X_train, maxlen=max_length, padding='post')
X_val = pad_sequences(X_val, maxlen=max_length, padding='post')

In [11]:
batch_size = 64

train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.long), torch.tensor(y_train, dtype=torch.long))
val_dataset = TensorDataset(torch.tensor(X_val, dtype=torch.long), torch.tensor(y_val, dtype=torch.long))

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [12]:
class SentimentLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super(SentimentLSTM, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=2, batch_first=True, dropout=0.5)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = self.embedding(x)
        x, _ = self.lstm(x)
        x = self.dropout(x[:, -1, :]) 
        x = self.fc(x)
        return x

In [13]:
vocab_size = 5000
embedding_dim = 128
hidden_dim = 256
output_dim = len(label_encoder.classes_)

In [14]:
model = SentimentLSTM(vocab_size, embedding_dim, hidden_dim, output_dim)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [15]:
epochs = 10  
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

for epoch in range(epochs):
    model.train()
    train_loss = 0
    
    for batch_idx, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()  
        outputs = model(inputs)  
        loss = criterion(outputs, labels)  
        
        loss.backward()  
        optimizer.step()  
        
        train_loss += loss.item()
        
        if batch_idx % 100 == 0:
            print(f"Epoch [{epoch+1}/{epochs}], Batch [{batch_idx}/{len(train_loader)}], Loss: {loss.item():.4f}")

    model.eval()
    val_preds, val_labels = [], []
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            val_preds.extend(predicted.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())
    
    val_accuracy = accuracy_score(val_labels, val_preds)
    print(f"Epoch {epoch + 1}/{epochs} | Train Loss: {train_loss / len(train_loader):.4f} | Validation Accuracy: {val_accuracy:.4f}")


print("Accuracy Score:", val_accuracy)
print("Classification Report:\n", classification_report(val_labels, val_preds, target_names=label_encoder.classes_))


Epoch [1/10], Batch [0/1157], Loss: 1.4019
Epoch [1/10], Batch [100/1157], Loss: 1.3771
Epoch [1/10], Batch [200/1157], Loss: 1.4088
Epoch [1/10], Batch [300/1157], Loss: 1.3580
Epoch [1/10], Batch [400/1157], Loss: 1.4071
Epoch [1/10], Batch [500/1157], Loss: 1.3708
Epoch [1/10], Batch [600/1157], Loss: 1.3546
Epoch [1/10], Batch [700/1157], Loss: 1.3982
Epoch [1/10], Batch [800/1157], Loss: 1.3981
Epoch [1/10], Batch [900/1157], Loss: 1.3917
Epoch [1/10], Batch [1000/1157], Loss: 1.3781
Epoch [1/10], Batch [1100/1157], Loss: 1.3828
Epoch 1/10 | Train Loss: 1.3685 | Validation Accuracy: 0.2663
Epoch [2/10], Batch [0/1157], Loss: 1.3315
Epoch [2/10], Batch [100/1157], Loss: 1.3706
Epoch [2/10], Batch [200/1157], Loss: 1.3771
Epoch [2/10], Batch [300/1157], Loss: 1.3551
Epoch [2/10], Batch [400/1157], Loss: 1.4094
Epoch [2/10], Batch [500/1157], Loss: 1.3517
Epoch [2/10], Batch [600/1157], Loss: 1.3645
Epoch [2/10], Batch [700/1157], Loss: 1.3720
Epoch [2/10], Batch [800/1157], Loss: 1.